In [1]:
import pandas as pd
import numpy as np

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlalchemy

from dateutil import parser
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta


In [2]:
database_username = 'smarsproddbuser@mbt-smars-mysql'
database_password = 'modern@1234'
database_ip       = 'mbt-smars-mysql.mysql.database.azure.com'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [3]:
df = pd.read_sql('SELECT * FROM mcrecos', con=database_connection)
#del df['srno']
del df['index']


In [4]:
df

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Cox and Kings,220.0,ICICI Direct,2018-11-23 03:06:00
1,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23 03:05:00
2,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23 03:03:00
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23 03:02:00
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23 02:54:00
...,...,...,...,...
5210,Samkrg Pistons,210.0,Firstcall,2016-01-05 10:39:00
5211,HDIL,83.0,Rajat Bose,2016-01-04 10:33:00
5212,DLF,125.5,Rajat Bose,2016-01-04 09:22:00
5213,Vinati Organics,555.0,Karvy,2016-01-01 12:37:00


In [5]:
df.drop_duplicates(inplace = True)
df

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Cox and Kings,220.0,ICICI Direct,2018-11-23 03:06:00
1,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23 03:05:00
2,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23 03:03:00
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23 03:02:00
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23 02:54:00
...,...,...,...,...
5210,Samkrg Pistons,210.0,Firstcall,2016-01-05 10:39:00
5211,HDIL,83.0,Rajat Bose,2016-01-04 10:33:00
5212,DLF,125.5,Rajat Bose,2016-01-04 09:22:00
5213,Vinati Organics,555.0,Karvy,2016-01-01 12:37:00


In [6]:
df['Call_Time_normal'] =  pd.to_datetime(df['Call_Time_normal'], format='%Y-%m-%d')
df['Call_Time_normal'] = df['Call_Time_normal'].dt.date
df

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Cox and Kings,220.0,ICICI Direct,2018-11-23
1,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23
2,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23
...,...,...,...,...
5210,Samkrg Pistons,210.0,Firstcall,2016-01-05
5211,HDIL,83.0,Rajat Bose,2016-01-04
5212,DLF,125.5,Rajat Bose,2016-01-04
5213,Vinati Organics,555.0,Karvy,2016-01-01


In [7]:
start_date = '2018-01-01'
end_date = '2018-12-31'
date_object1 = datetime.strptime(start_date, '%Y-%m-%d').date()
date_object2 = datetime.strptime(end_date, '%Y-%m-%d').date()

In [8]:
mask = (df['Call_Time_normal'] > date_object1) & (df['Call_Time_normal'] <= date_object2)

# 2018 data

In [9]:
df1 = df.loc[mask]
df1 = df1.reset_index()
#del df1['level_0']
#del df1['srno']
del df1['index']

In [10]:
df1

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Cox and Kings,220.0,ICICI Direct,2018-11-23
1,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23
2,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23
...,...,...,...,...
2113,Reliance Industries,1151.0,Edelweiss,2018-01-03
2114,Century Plyboards,398.0,Kotak Securities,2018-01-03
2115,ICICI Bank,370.0,Motilal Oswal,2018-01-03
2116,NMDC,195.0,Edelweiss,2018-01-03


In [11]:
print("No of Brokers in 2016 :",df1['Analyst_or_Cmp'].nunique()) 
s1=len(df1)

No of Brokers in 2016 : 95


## CLEANING

In [12]:
a = pd.DataFrame()
for i in range(0,len(df1)): 
    sym=df1.at[i, 'Cmp_Name']
    sym = sym.lower()
    sym = sym.replace(" ","")
    sym = sym.replace(".","")
    sym = sym.replace("(i)","")
    sym = sym.replace("(","")
    sym = sym.replace(")","")
    sym = sym.replace("-","")
    sym = sym.replace(",","")
    sym = sym.replace('see23%cagr',"")
    sym = sym.replace("drreddy'slaboratories",'drreddyslabs')
    print(sym)
    sqlrun = "SELECT SYMBOL FROM symbolsmappedmc WHERE recoName = '"+sym+"'"
    a = a.append(pd.read_sql(sqlrun, con=database_connection),ignore_index=True)

coxandkings
igpetrochemicals
siyaramsilkmills
jklakshmicement
sunteckrealty
rupaandcompany
himadrispeciality
shailyengineeringplastics
shreepushkarchemicals
natcopharma
marutisuzukiindia
coromandelinternational
ashokleyland
indianbank
mirzainternational
aartidrugs
nbcc
cesc
thermax
vardhmantextiles
unitedbreweries
bankofbaroda
ahluwaliacontractsindia
jklakshmicement
finolexcables
sanghiindustries
pncinfratech
sunteckrealty
apollotyres
starcement
deccancements
nclindustries
mindtree
federalbank
pncinfratech
vatechwabag
repcohomefinance
suprajitengineering
jbmauto
sadbhavengineering
lemontree
ahluwaliacontracts
jklakshmicements
raymond
vedanta
reliancecommunications
filatexindia
sanghiindustries
mahanagargas
ashokleyland
sterlingtools
pncinfra
marutisuzuki
jindalsteelandpower
nmdc
ashokleyland
relianceindustries
larsen&toubro
aurobindopharma
nrbbearings
kaveriseedco
mahindraandmahindra
sunpharma
ashokleyland
ipcalaboratories
mahindraandmahindra
futureretail
maxfinancialservices
canaraban

rblbank
techmahindra
aurobindopharma
timkenindia
sadbhavengineering
maharashtraseamless
cochinshipyard
sunpharmaceuticals
hindalco
statebankofindia
lupin
fiemindustries
exideindustries
suntvnetwork
ppapautomotive
l&ttechnologyservices
m&mfinancialservices
sundramfasteners
sonatasoftware
sadbhavengineering
ashokabuildcon
oilindia
knrconstructions
statebankofindia
neulandlabs
zensartechnologies
alkemlabs
indraprasthagas
aurobindopharma
futureconsumer
nalco
nmdc
kaveriseedco
stridesshasun
zensartechnologies
vatechwabag
tatacommunications
trident
indiagridtrust
indianbank
mothersonsumisystem
arvind
piindustries
jkumarinfraprojects
gulfoillubricants
statebankofindia
zensartechnologies
rainindustries
koltepatildevelopers
zentechnologies
radicokhaitan
hdfcstandardlife
ineosstyrolution
aiaengineering
trent
nrbbearings
pncinfratech
varunbeverage
bharatforge
deccancements
ppapautomotive
mirzainternational
cipla
bharatagrifertandrealty
indusindbank
pidiliteindustries
gulfoillubricants
jmcprojects

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'slabs'' at line 1
[SQL: SELECT SYMBOL FROM symbolsmappedmc WHERE recoName = 'drreddy'slabs']
(Background on this error at: http://sqlalche.me/e/f405)

In [13]:
len(df1)

2118

In [14]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850 entries, 0 to 849
Data columns (total 1 columns):
SYMBOL    604 non-null object
dtypes: object(1)
memory usage: 6.8+ KB


In [15]:
a["SYMBOL"].fillna(method='ffill', inplace = True) 

In [16]:
a=a.replace("None",np.nan)

In [17]:
a

,SYMBOL
0,None
1,None
2,None
3,SIYSIL
4,JKLAKSHMI
...,...
845,SUNTV
846,SUNTV
847,JETAIRWAYS
848,TECHM


## SYMBOLS ADDED TO TABLE

In [18]:
df4 =pd.concat([df1,a],axis=1)

In [19]:
df4 = df4.dropna()

In [20]:
df4

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal,SYMBOL
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23,SIYSIL
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23,JKLAKSHMI
5,Rupa and Company,370.0,ICICI Direct,2018-11-23,JKLAKSHMI
6,Himadri Speciality,160.0,ICICI Direct,2018-11-23,SUNTECK
7,Shaily Engineering Plastics,1120.0,ICICI Direct,2018-11-23,SUNTECK
...,...,...,...,...,...
845,Symphony,1710.0,KRChoksey,2018-07-31,SUNTV
846,Colgate-Palmolive India,1321.0,KRChoksey,2018-07-31,SUNTV
847,Jammu and Kashmir Bank,94.0,Prabhudas Lilladher,2018-07-31,JETAIRWAYS
848,Petronet LNG,300.0,Prabhudas Lilladher,2018-07-31,TECHM


In [21]:
uniqueBrokers = df4.Analyst_or_Cmp.unique()
uniqueBrokers

array(['ICICI Direct', 'Geojit', 'Nalanda Securities', 'SPA Securities',
       'Anand Rathi', 'Bonanza', 'Rupak De\u200b', 'Kotak Securities',
       'HDFC Securities', 'Cholamandalam Securities', 'Arpan Shah',
       'Shitij Gandhi', 'Dalmia Securities', 'Dolat Capital',
       'Motilal Oswal', 'Ashish Chaturmohta', 'LKP Research',
       'Prabhudas Lilladher', 'Vinay Rajani', 'Shabbir Kayyumi',
       'Dharmesh Shah', 'Centrum', 'Rupak De', 'KR Choksey',
       'Rudra Shares & Stock Brokers', 'Dinesh Rohira',
       'Hadrien Mendonca', 'Aditya Agarwala', 'Abhishek Mondal',
       'JM Financial', 'Sharekhan', 'Akash Jain', 'Jayant Manglik',
       'Cholamandalam securities', 'Choice Equity', 'Mazhar Mohammad',
       'Sameet Chavan', 'Arihant Capital', 'Emkay Global',
       'Mustafa Nadeem', 'Nandish Shah', 'Siddharth Sedani',
       'Rudra Shares', 'CD Equisearch', 'Manav Chopra', 'Aditya Agarwal',
       'Khambatta Securities', 'AnandRathi', 'Rajesh Palviya',
       'Joindre Capit

In [22]:
#uniqueBrokers = uniqueBrokers[:1]

## PERFORMANCE DEDUCTION ALGORITHM

In [23]:
#del brokerPerformance1
brokerPerformance1 = pd.DataFrame(columns=['Name','Hit Ratio','Avg. Growth','Total Recos'])
brokerPerformance1

,Name,Hit Ratio,Avg. Growth,Total Recos


In [24]:
lost =[]
for brokerName in uniqueBrokers :
    dfbrokerRecos = df4.loc[df4['Analyst_or_Cmp'] == brokerName]
    print('brokerName : ',brokerName)
    hits=0
    growthSum=0
    total=0
    for index, row in dfbrokerRecos.iterrows():
        #print(row)
        sym = row['SYMBOL']
        print(sym)

        avgGrowth=0
        targetp=0
        recbuying=0
        maxval=0
        growthperc=0
        
        if type(sym) == str :
            mydate1 = row['Call_Time_normal']#prediction date
            mydate2 = mydate1 + relativedelta(years=1)#date 1 year after prediction
            dbyear1 = "nse"+str(mydate1.year)
            dbyear2 = "nse"+str(mydate2.year)
            print(mydate1)
            
            #Fetching current price from nse bav copy
            cnx = mysql.connector.connect(user="root", password='yash', host="127.0.0.1", port=3306, database='mainschema')
            sql_select_Query = "SELECT Closeval FROM "+dbyear1+" WHERE Date = '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
            cursor = cnx.cursor()
            cursor.execute(sql_select_Query)
            record = cursor.fetchall()
            if len(record) != 0:
                
                x = int(record[0][0])
            
        
                sqlrun1 = "SELECT * FROM "+dbyear1+" WHERE Date >= '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
                dfprices1 = pd.read_sql(sqlrun1, con=database_connection)
                del dfprices1['index']
        
                sqlrun2 = "SELECT * FROM "+dbyear2+" WHERE Date <= '"+str(mydate2)+"' AND Symbol = '"+sym+"'"
                dfprices2 = pd.read_sql(sqlrun2, con=database_connection)
                del dfprices2['index']
                dfprices = pd.concat([dfprices1,dfprices2],ignore_index=True)

                targetp = float(row['Target_Price'])
        
                print("SYMBOL : ",sym)
                dfhits = dfprices.loc[dfprices['Closeval'] >= targetp]
            
                total = total + 1
            
                if not dfhits.empty :
                    print("HIT!")
                    hits = hits + 1
                    #recbuying = float(row['recomended_buying'])
                    maxval = dfprices['Closeval'].max()
                    growthperc = (maxval - x) *100 / x
                    print('Growth : ',growthperc)
                    growthSum = growthSum + growthperc
                    
                
                else:
                    print("DIDN'T HIT!")
                del dfprices
                del dfhits
            else:
                lost.append(sym)
    if total != 0: 
        hitRatio = int(hits*100 / total)
        print("Growthsum",growthSum)
        avgGrowth = int(growthSum / total)
        print("Broker : ", brokerName, "HitRatio : ", hitRatio, "Avg. Growth : ", avgGrowth, "Total Recos. : ", total)
        brokerPerformance1 = brokerPerformance1.append({'Name':brokerName,'Hit Ratio':hitRatio,'Avg. Growth':avgGrowth,'Total Recos':total}, ignore_index=True)

brokerName :  ICICI Direct
SIYSIL
2018-11-23
JKLAKSHMI
2018-11-23
JKLAKSHMI
2018-11-23
SUNTECK
2018-11-23
SUNTECK
2018-11-23
HSCL
2018-11-23
MIRZAINT
2018-11-22
SYMBOL :  MIRZAINT
HIT!
Growth :  10.0
AARTIDRUGS
2018-11-22
SYMBOL :  AARTIDRUGS
DIDN'T HIT!
NBCC
2018-11-22
SYMBOL :  NBCC
DIDN'T HIT!
NBCC
2018-11-22
SYMBOL :  NBCC
DIDN'T HIT!
THERMAX
2018-11-22
SYMBOL :  THERMAX
DIDN'T HIT!
MARICO
2018-11-14
SYMBOL :  MARICO
DIDN'T HIT!
MCDOWELL-N
2018-11-14
SYMBOL :  MCDOWELL-N
HIT!
Growth :  3.925811437403397
HEXAWARE
2018-11-14
SYMBOL :  HEXAWARE
DIDN'T HIT!
THYROCARE
2018-11-14
SYMBOL :  THYROCARE
DIDN'T HIT!
FCONSUMER
2018-11-13
SYMBOL :  FCONSUMER
DIDN'T HIT!
TATACOMM
2018-11-13
SYMBOL :  TATACOMM
HIT!
Growth :  19.7093023255814
SUNTV
2018-11-13
SYMBOL :  SUNTV
HIT!
Growth :  5.040916530278225
GUJGASLTD
2018-11-13
SYMBOL :  GUJGASLTD
DIDN'T HIT!
CADILAHC
2018-11-13
SYMBOL :  CADILAHC
DIDN'T HIT!
ASHOKA
2018-11-13
SYMBOL :  ASHOKA
DIDN'T HIT!
ASHOKA
2018-11-13
SYMBOL :  ASHOKA
DIDN'T 

SYMBOL :  BANKBARODA
DIDN'T HIT!
AHLUCONT
2018-11-22
SYMBOL :  AHLUCONT
DIDN'T HIT!
JKLAKSHMI
2018-11-22
SYMBOL :  JKLAKSHMI
HIT!
Growth :  45.5925925925926
JKLAKSHMI
2018-11-22
SYMBOL :  JKLAKSHMI
HIT!
Growth :  45.5925925925926
FINCABLES
2018-11-22
SYMBOL :  FINCABLES
DIDN'T HIT!
FINCABLES
2018-11-22
SYMBOL :  FINCABLES
HIT!
Growth :  7.9870129870129825
PNCINFRA
2018-11-22
SYMBOL :  PNCINFRA
HIT!
Growth :  36.70967741935485
SUNTECK
2018-11-22
SYMBOL :  SUNTECK
DIDN'T HIT!
APOLLOTYRE
2018-11-22
SYMBOL :  APOLLOTYRE
HIT!
Growth :  9.931192660550462
INDHOTEL
2018-11-16
SYMBOL :  INDHOTEL
DIDN'T HIT!
INDHOTEL
2018-11-16
SYMBOL :  INDHOTEL
DIDN'T HIT!
ORIENTCEM
2018-11-16
SYMBOL :  ORIENTCEM
DIDN'T HIT!
INSECTICID
2018-11-15
SYMBOL :  INSECTICID
DIDN'T HIT!
GULFOILLUB
2018-11-15
SYMBOL :  GULFOILLUB
HIT!
Growth :  19.821428571428573
JAMNAAUTO
2018-11-15
SYMBOL :  JAMNAAUTO
DIDN'T HIT!
JAMNAAUTO
2018-11-15
SYMBOL :  JAMNAAUTO
DIDN'T HIT!
VOLTAMP
2018-11-15
SYMBOL :  VOLTAMP
HIT!
Growth :  

SYMBOL :  SUNTECK
HIT!
Growth :  6.909650924024636
AHLUCONT
2018-08-21
SYMBOL :  AHLUCONT
DIDN'T HIT!
REPCOHOME
2018-08-21
SYMBOL :  REPCOHOME
DIDN'T HIT!
L&TFH
2018-08-17
SYMBOL :  L&TFH
DIDN'T HIT!
AMARAJABAT
2018-08-17
SYMBOL :  AMARAJABAT
HIT!
Growth :  4.060676779463239
AMARAJABAT
2018-08-17
SYMBOL :  AMARAJABAT
HIT!
Growth :  4.060676779463239
TECHM
2018-08-17
SYMBOL :  TECHM
HIT!
Growth :  21.83406113537118
AUROPHARMA
2018-08-17
SYMBOL :  AUROPHARMA
HIT!
Growth :  26.89312977099236
TIMKEN
2018-08-17
SYMBOL :  TIMKEN
HIT!
Growth :  4.012693935119893
SADBHAV
2018-08-17
SYMBOL :  SADBHAV
DIDN'T HIT!
MAHSEAMLES
2018-08-17
SYMBOL :  MAHSEAMLES
DIDN'T HIT!
COCHINSHIP
2018-08-17
SYMBOL :  COCHINSHIP
DIDN'T HIT!
COCHINSHIP
2018-08-17
SYMBOL :  COCHINSHIP
HIT!
Growth :  0.0
GULFOILLUB
2018-08-10
SYMBOL :  GULFOILLUB
DIDN'T HIT!
GULFOILLUB
2018-08-10
SYMBOL :  GULFOILLUB
HIT!
Growth :  3.644164759725403
ZENSARTECH
2018-08-10
SYMBOL :  ZENSARTECH
HIT!
Growth :  35.508064516129025
ZENSARTEC

SYMBOL :  ENIL
DIDN'T HIT!
SONATSOFTW
2018-11-14
SYMBOL :  SONATSOFTW
DIDN'T HIT!
GUJGASLTD
2018-11-14
SYMBOL :  GUJGASLTD
DIDN'T HIT!
PETRONET
2018-11-14
SYMBOL :  PETRONET
DIDN'T HIT!
PETRONET
2018-11-14
SYMBOL :  PETRONET
DIDN'T HIT!
GRAPHITE
2018-11-14
SYMBOL :  GRAPHITE
HIT!
Growth :  0.08008008008007553
SRF
2018-11-14
SYMBOL :  SRF
HIT!
Growth :  54.584148727984335
BHARATFORG
2018-11-13
SYMBOL :  BHARATFORG
DIDN'T HIT!
TCIEXP
2018-11-12
SYMBOL :  TCIEXP
HIT!
Growth :  29.836867862968997
IGL
2018-11-12
SYMBOL :  IGL
HIT!
Growth :  55.218181818181826
TITAN
2018-11-12
SYMBOL :  TITAN
HIT!
Growth :  53.752796420581646
VOLTAS
2018-11-12
SYMBOL :  VOLTAS
DIDN'T HIT!
IGL
2018-11-12
SYMBOL :  IGL
HIT!
Growth :  55.218181818181826
KHADIM
2018-11-12
SYMBOL :  KHADIM
DIDN'T HIT!
BEL
2018-10-30
SYMBOL :  BEL
DIDN'T HIT!
ICICIBANK
2018-10-26
SYMBOL :  ICICIBANK
HIT!
Growth :  48.920634920634924
RADIOCITY
2018-10-26
SYMBOL :  RADIOCITY
DIDN'T HIT!
CROMPTON
2018-10-26
SYMBOL :  CROMPTON
HIT!
Gr

DHAMPURSUG
2018-10-09
SYMBOL :  DHAMPURSUG
DIDN'T HIT!
NATCOPHARM
2018-09-07
SYMBOL :  NATCOPHARM
DIDN'T HIT!
FLUOROCHEM
2018-08-30
NESTLEIND
2018-08-21
SYMBOL :  NESTLEIND
HIT!
Growth :  15.69079551717803
BRIGADE
2018-08-21
SYMBOL :  BRIGADE
DIDN'T HIT!
REPCOHOME
2018-08-21
SYMBOL :  REPCOHOME
DIDN'T HIT!
ZENSARTECH
2018-08-14
SYMBOL :  ZENSARTECH
HIT!
Growth :  33.040380047505934
ALKEM
2018-08-14
SYMBOL :  ALKEM
HIT!
Growth :  9.514925373134329
VBL
2018-08-10
SYMBOL :  VBL
HIT!
Growth :  27.496697490092465
PSPPROJECT
2018-08-08
SYMBOL :  PSPPROJECT
HIT!
Growth :  14.358178053830231
MPHASIS
2018-08-08
SYMBOL :  MPHASIS
HIT!
Growth :  10.430809399477798
HINDPETRO
2018-08-08
SYMBOL :  HINDPETRO
DIDN'T HIT!
SOBHA
2018-08-08
SYMBOL :  SOBHA
HIT!
Growth :  16.861167002012063
DEEPAKNTR
2018-08-02
SYMBOL :  DEEPAKNTR
DIDN'T HIT!
JETAIRWAYS
2018-07-31
SYMBOL :  JETAIRWAYS
HIT!
Growth :  12.135922330097088
TECHM
2018-07-31
SYMBOL :  TECHM
HIT!
Growth :  23.08823529411765
KALPATPOWR
2018-07-31


SYMBOL :  TORNTPOWER
DIDN'T HIT!
ICICIPRULI
2018-10-23
SYMBOL :  ICICIPRULI
HIT!
Growth :  50.216718266253864
ICICIPRULI
2018-10-23
SYMBOL :  ICICIPRULI
DIDN'T HIT!
ICICIPRULI
2018-10-23
SYMBOL :  ICICIPRULI
DIDN'T HIT!
COLPAL
2018-10-23
SYMBOL :  COLPAL
HIT!
Growth :  37.462555066079304
COLPAL
2018-10-23
SYMBOL :  COLPAL
HIT!
Growth :  37.462555066079304
Growthsum 157.55408240531042
Broker :  Rudra Shares & Stock Brokers HitRatio :  55 Avg. Growth :  17 Total Recos. :  9
brokerName :  Dinesh Rohira
TORNTPOWER
2018-11-05
SYMBOL :  TORNTPOWER
HIT!
Growth :  17.358490566037737
ASHOKA
2018-09-19
SYMBOL :  ASHOKA
DIDN'T HIT!
HSCL
2018-09-05
SYMBOL :  HSCL
DIDN'T HIT!
AUROPHARMA
2018-09-05
SYMBOL :  AUROPHARMA
HIT!
Growth :  19.41810344827586
JKLAKSHMI
2018-08-06
SYMBOL :  JKLAKSHMI
HIT!
Growth :  22.461059190031158
JKLAKSHMI
2018-08-06
SYMBOL :  JKLAKSHMI
HIT!
Growth :  22.461059190031158
Growthsum 81.69871239437592
Broker :  Dinesh Rohira HitRatio :  66 Avg. Growth :  13 Total Recos. :  6

brokerName :  Mustafa Nadeem
FEDERALBNK
2018-10-17
SYMBOL :  FEDERALBNK
DIDN'T HIT!
COCHINSHIP
2018-10-17
SYMBOL :  COCHINSHIP
HIT!
Growth :  4.276485788113698
VISAKAIND
2018-09-28
SYMBOL :  VISAKAIND
HIT!
Growth :  15.656108597285066
Growthsum 19.932594385398765
Broker :  Mustafa Nadeem HitRatio :  66 Avg. Growth :  6 Total Recos. :  3
brokerName :  Nandish Shah
HEROMOTOCO
2018-10-16
SYMBOL :  HEROMOTOCO
HIT!
Growth :  16.108808290155437
INDOSTAR
2018-10-16
SYMBOL :  INDOSTAR
HIT!
Growth :  37.17741935483871
INDUSINDBK
2018-10-16
SYMBOL :  INDUSINDBK
HIT!
Growth :  12.413580246913575
NTPC
2018-09-25
SYMBOL :  NTPC
HIT!
Growth :  1097.0588235294117
RELCAPITAL
2018-09-11
SYMBOL :  RELCAPITAL
DIDN'T HIT!
PEL
2018-09-11
SYMBOL :  PEL
HIT!
Growth :  4123.9904175222455
ASTERDM
2018-08-21
SYMBOL :  ASTERDM
DIDN'T HIT!
ASTERDM
2018-08-21
SYMBOL :  ASTERDM
DIDN'T HIT!
ASTERDM
2018-08-21
SYMBOL :  ASTERDM
DIDN'T HIT!
TIMETECHNO
2018-08-07
SYMBOL :  TIMETECHNO
DIDN'T HIT!
NTPC
2018-08-07
SYMBOL 

In [25]:
len(lost)

52

In [26]:
brokerPerformance1

,Name,Hit Ratio,Avg. Growth,Total Recos
0,ICICI Direct,47,24,92
1,Geojit,0,0,1
2,Nalanda Securities,42,7,28
3,SPA Securities,100,7,1
4,Anand Rathi,47,13,38
5,Bonanza,50,17,2
6,Rupak De​,100,25,1
7,Kotak Securities,53,471,30
8,HDFC Securities,51,136,100
9,Cholamandalam Securities,44,34,29


In [30]:
brokerPerformance1.to_sql(con=database_connection, name='MCbrokerperformance2018', if_exists='replace')

C:\Users\dhruv\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\io\sql.py:1336: UserWarning: The provided table name 'MCbrokerperformance2017' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
